In [6]:
from minio import Minio
import os

In [7]:
# Create client with access key and secret key with specific region.
client1 = Minio(
    "minio-0:9000",
    access_key="admin",
    secret_key="adminkey",
    secure=False
)

client2 = Minio(
    "minio-1:9000",
    access_key="admin",
    secret_key="adminkey",
    secure=False
)

client3 = Minio(
    "minio-2:9000",
    access_key="admin",
    secret_key="adminkey",
    secure=False
)


# # Create client with custom HTTP client using proxy server.
# import urllib3
# client = Minio(
#     "SERVER:PORT",
#     access_key="ACCESS_KEY",
#     secret_key="SECRET_KEY",
#     secure=True,
#     http_client=urllib3.ProxyManager(
#         "https://PROXYSERVER:PROXYPORT/",
#         timeout=urllib3.Timeout.DEFAULT_TIMEOUT,
#         cert_reqs="CERT_REQUIRED",
#         retries=urllib3.Retry(
#             total=5,
#             backoff_factor=0.2,
#             status_forcelist=[500, 502, 503, 504],
#         ),
#     ),
#)

### Download data from client1 to workspace for performance testing

In [3]:
%time
buckets = client1.list_buckets()
for b,bucket in enumerate(buckets):
    print(b, "Bucket: " ,bucket.name, bucket.creation_date)
    os.makedirs(bucket.name,exist_ok=True)
    # List objects information.
    objects = client1.list_objects(bucket.name)
    for o,obj in enumerate(objects):
        #print("\t",o, "Object: ", obj.object_name)
        # Download data of an object.
        client1.fget_object(bucket.name,obj.object_name, os.path.join(bucket.name,obj.object_name))

CPU times: user 12 µs, sys: 3 µs, total: 15 µs
Wall time: 27.9 µs
0 Bucket:  dummy 2022-03-04 05:02:33.664000+00:00
1 Bucket:  test2 2022-03-04 01:30:18.221000+00:00


### Upload data from workspace to client1, dummy bucket

In [15]:
from minio.deleteobjects import DeleteObject
def remove_nonempty_bucket(client,bucket_name):
    if client.bucket_exists(bucket_name):
        if len(list(client.list_objects(bucket_name))) > 0:
            delete_object_list = map(lambda x: DeleteObject(x.object_name),
                                     client.list_objects(bucket_name,recursive=True),)
            errors = client.remove_objects(bucket_name, delete_object_list)
            for error in errors:
                print("error occured when deleting object", error)
        client.remove_bucket(bucket_name)
        print("Bucket ", bucket_name, " removed!")
    else:
        print("Bucket ", bucket_name, " does not exist!")

In [19]:
%%time
# Upload data with metadata.
#from minio_utils.progress import Progress
new_bucket = "dummy"
upload_folder = "test2/"

client = client3

remove_nonempty_bucket(client,new_bucket)

client.make_bucket(new_bucket)

files = os.listdir(upload_folder)

for f in files:
    prefix = f[:2]
    result = client.fput_object(
    new_bucket, f, os.path.join(upload_folder,f),
    metadata={"prefix": prefix},
    )
    #print("created {0} object; etag: {1}, version-id: {2}".format(result.object_name, result.etag, result.version_id,),)
print("Upload Done")
# # Upload data with tags, retention and legal-hold.
# date = datetime.utcnow().replace(
#     hour=0, minute=0, second=0, microsecond=0,
# ) + timedelta(days=30)
# tags = Tags(for_object=True)
# tags["User"] = "jsmith"
# result = client.fput_object(
#     "my-bucket", "my-object", "my-filename",
#     tags=tags,
#     retention=Retention(GOVERNANCE, date),
#     legal_hold=True,
# )
# print(
#     "created {0} object; etag: {1}, version-id: {2}".format(
#         result.object_name, result.etag, result.version_id,
#     ),
# )


Bucket  dummy  removed!
Upload Done
CPU times: user 499 ms, sys: 67.6 ms, total: 566 ms
Wall time: 1.37 s
